In [19]:
from langchain.vectorstores import Chroma
from langchain.document_loaders import WikipediaLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from dotenv import load_dotenv
load_dotenv()
import os, warnings
warnings.filterwarnings('ignore')
from langchain.chat_models import ChatOpenAI
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [21]:
api_key = os.environ['OPENAI_API_KEY']

In [23]:
embeddings = OpenAIEmbeddings(api_key=api_key)

In [25]:
llm = ChatOpenAI(api_key=api_key,temperature=0)

In [45]:
db_connection = Chroma(persist_directory='some_new_mkultra/',embedding_function=embeddings)

In [47]:
compressor = LLMChainExtractor.from_llm(llm)

In [49]:
# Contextual compression
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor,base_retriever=db_connection.as_retriever())

In [53]:
docs = db_connection.similarity_search("When was this declassified?")
docs[0].page_content

'The Church Committee (formally the United States Senate Select Committee to Study Governmental Operations with Respect to Intelligence Activities) was a US Senate select committee in 1975 that  investigated abuses by the Central Intelligence Agency (CIA), National Security Agency (NSA), Federal Bureau of Investigation (FBI), and the Internal Revenue Service (IRS). Chaired by Idaho Senator Frank Church (D-ID), the committee was part of a series of investigations into intelligence abuses in 1975, dubbed the "Year of Intelligence", including its House counterpart, the Pike Committee, and the presidential Rockefeller Commission. The committee\'s efforts led to the establishment of the permanent US Senate Select Committee on Intelligence.\nThe most shocking revelations of the committee include Operation MKULTRA, which involved the drugging and torture of unwitting US citizens as part of human experimentation on mind control; COINTELPRO, which involved the surveillance and infiltration of A

In [55]:
compressed_docs = compression_retriever.get_relevant_documents(query="When was this declassified")

In [57]:
compressed_docs[0].page_content

'The creation of the Church Committee was approved on January 27, 1975, by a vote of 82 to 4 in the Senate.'

In [59]:
compressed_docs[0].metadata['summary']

'The Church Committee (formally the United States Senate Select Committee to Study Governmental Operations with Respect to Intelligence Activities) was a US Senate select committee in 1975 that  investigated abuses by the Central Intelligence Agency (CIA), National Security Agency (NSA), Federal Bureau of Investigation (FBI), and the Internal Revenue Service (IRS). Chaired by Idaho Senator Frank Church (D-ID), the committee was part of a series of investigations into intelligence abuses in 1975, dubbed the "Year of Intelligence", including its House counterpart, the Pike Committee, and the presidential Rockefeller Commission. The committee\'s efforts led to the establishment of the permanent US Senate Select Committee on Intelligence.\nThe most shocking revelations of the committee include Operation MKULTRA, which involved the drugging and torture of unwitting US citizens as part of human experimentation on mind control; COINTELPRO, which involved the surveillance and infiltration of A